For now just play with sine waves

In [10]:
# Imports
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal as sg
from math import sin, pi

import alsaaudio as aa

import time
import struct
import itertools
from multiprocessing import Queue
from queue import Empty
from threading import Thread

In [11]:
# Initialization of some values
precision = 24
freq = 500
duration = 30               # In seconds

channels = 2                # This is practically the only possibility as far as I managed to understand :|
rate = 192000               # Maximum rate supported by the board
length = 0.1                # Length in time of the signal we create
amp = ((2**precision)/2-1)  

In [12]:
def byte_conversion(data_list):
    '''Converts a list of integers into a list of bytes with specified length'''

    if precision == 16:
        num_bytes = 2
        bytes_array = [(integer).to_bytes(num_bytes, byteorder='little', signed=True) for integer in data_list]
        bytes_arr = bytearray()
        for b in bytes_array:
            bytes_arr.extend(b)
        return bytes(bytes_arr)
    
    elif precision == 24:
        num_bytes = 24
        bytes_array = [(integer).to_bytes(num_bytes, byteorder='little', signed=True) for integer in data_list]
        bytes_arr = bytearray()
        for b in bytes_array:
            bytes_arr.extend(b)
        return bytes(bytes_arr)
    

def generate_sin(frequency, amplitude, duration = length):
    '''Generates a sinusoidal function with specified parametrs. 
    It returns a bytes list ready to be sent to the board'''

    cycle_size = int(rate / frequency)
    factor = int(duration * frequency)

    # Total number of frames
    frames = cycle_size * factor

    signal = [int(amplitude * sin(2 * pi * frequency * i / rate)) for i in range(frames)]

    if channels > 1:
        signal = list(itertools.chain.from_iterable(itertools.repeat(x, channels) for x in signal))

    return byte_conversion(signal)

In [13]:
class SinePlayer(Thread):
    '''Class to play a sinusoidal wave.'''
    def __init__(self, frequency = freq, duration = length, amplitude = amp):
        Thread.__init__(self, daemon=True)
        if precision == 16:
            self.device = aa.PCM(rate=rate, channels=channels, format=aa.PCM_FORMAT_S16_LE, periodsize=512, cardindex=1)
        elif precision == 24:
            self.device = aa.PCM(rate=rate, channels=channels, format=aa.PCM_FORMAT_S24_LE, periodsize=512, cardindex=1)
        elif precision == 32:
            self.device = aa.PCM(rate=rate, channels=channels, format=aa.PCM_FORMAT_S32_LE, periodsize=512, cardindex=1)
        else:
            raise ValueError('Precision specified is not 16, 24 or 32 bits.')
        # We open the device
        # Defaul settings are:
        #   - type = PCM_PLAYBACK
        #   - mode = PCM_NORMAL  -->  will block the 'write' function if the buffer is full
        #   - periods = 4

        self.queue = Queue()
        self.change(frequency, amplitude, duration)

    def change(self, frequency, amplitude, duration = length):
        '''Generate the buffer with the data and put it in the queue.'''

        #print('Ciao I\'m in the change function')

        if frequency > rate / 2:
            raise ValueError('maximum frequency is %d' % (rate / 2))

        buf = generate_sin(frequency, amplitude, duration)
        #print('len buf:', len(buf))
        self.queue.put(buf)
        #print('putted the data in the queue')

    def run(self):
        #print('Entered the run function')
        buffer = None
        while True:
            try:
                buffer = self.queue.get(False)
                #print('len buffer:', len(buffer))
                #print(buffer)
                print('Got the queued data, data stream should start :)')
            except Empty:
                pass
            if buffer:
                if self.device.write(buffer) < 0:
                    print("Playback buffer underrun! Continuing nonetheless ...")

In [14]:
isine = SinePlayer()
isine.start() 

Got the queued data, data stream should start :)


In [15]:
isine.device.info()

{'name': 'hw:1',
 'card_no': 1,
 'device_no': 0,
 'subdevice_no': 0,
 'state': 'RUNNING',
 'access_type': 'RW_INTERLEAVED',
 ' (call value) type': 0,
 ' (call value) type_name': 'PLAYBACK',
 ' (call value) mode': 0,
 ' (call value) mode_name': 'PCM_NORMAL',
 'format': 6,
 'format_name': 'S24_LE',
 'format_description': 'Signed 24 bit Little Endian',
 'subformat_name': 'STD',
 'subformat_description': 'Standard',
 'channels': 2,
 'rate': 192000,
 'period_time': 2666,
 'period_size': 512,
 'buffer_time': 10666,
 'buffer_size': 2048,
 'periods': 4,
 'rate_numden': (192000, 1),
 'significant_bits': 32,
 'is_batch': False,
 'is_block_transfer': False,
 'is_double': False,
 'is_half_duplex': False,
 'is_joint_duplex': False,
 'can_overrange': False,
 'can_mmap_sample_resolution': True,
 'can_pause': True,
 'can_resume': True,
 'can_sync_start': False}

In [16]:
time.sleep(duration) 

In [17]:
isine.device.close()

Exception in thread Thread-44:
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/tmp/ipykernel_7910/1580063620.py", line 47, in run
alsaaudio.ALSAAudioError: File descriptor in bad state [hw:1]
